# Notebook 3 — Testing & Evaluation

Loads the trained LightGBM model and produces final test metrics, including confusion matrix and per-class scores.

## Procedure

1. Reload the YAML config and processed parquet.
2. Split data identically to the training notebook to obtain the test partition.
3. Load the saved `lightgbm_pso.pkl` model.
4. Compute classification report, confusion matrix, and visualize results.
5. Persist test metrics to `artifacts/metrics/test_report.json`.

In [ ]:
from pathlib import Path
import sys
import json

import joblib
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
from src.config import DEFAULT_CONFIG_PATH, load_config

config = load_config(DEFAULT_CONFIG_PATH)
processed_path = Path(config.data.processed_file)
model_path = config.paths.models_path / "lightgbm_pso.pkl"

if not processed_path.exists() or not model_path.exists():
    raise FileNotFoundError("Ensure processed data and trained model exist before running evaluation.")

df = pd.read_parquet(processed_path)
label_col = config.data.label_column
X = df.drop(columns=[label_col])
y = df[label_col]

_, X_test, _, y_test = train_test_split(
    X,
    y,
    test_size=config.training.test_size,
    stratify=y,
    random_state=config.training.random_state,
)

test_sample = X_test.iloc[:5]
test_sample

In [ ]:
model = joblib.load(model_path)
preds = model.predict(X_test)

report = classification_report(y_test, preds, output_dict=True)
cm = confusion_matrix(y_test, preds)

print("Test macro F1:", report["macro avg"]["f1-score"])
report

In [ ]:
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.ylabel("True")
plt.xlabel("Predicted")
plt.show()

In [ ]:
metrics = {
    "classification_report": report,
    "confusion_matrix": cm.tolist(),
}
metrics_path = config.paths.metrics_path / "test_report.json"
metrics_path.write_text(json.dumps(metrics, indent=2), encoding="utf-8")
print(f"Saved metrics to {metrics_path}")

🎯 **Outcome**: Test metrics are now logged under `artifacts/metrics/test_report.json`. Share plots/confusion matrix screenshots in your final report.